In [1]:
#setting logging information

In [2]:
#We can track events in a software application by logging. Major reason why logging is better
#than printing is that the log record, which is created with every logging event, contains
#readily available diagnostic information

In [20]:
import logging
import warnings
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [4]:
#example
logging.warning('This is a warning!')

2018-06-18 15:11:15,173 : WARNING : This is a warning!


In [5]:
#download nltk stopwords and spacy model
#the exclamation before the command means that this statement will be
#executed in the cmd prompt

In [6]:
! pip install gensim

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [7]:
!pip install nltk

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [8]:
!pip install pandas

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [9]:
!pip install pyLDAvis

    100% |████████████████████████████████| 1.6MB 804kB/s ta 0:00:01
    100% |████████████████████████████████| 184kB 3.1MB/s ta 0:00:01
    100% |████████████████████████████████| 829kB 1.3MB/s ta 0:00:01
  Running setup.py bdist_wheel for pyLDAvis ... done
  Stored in directory: /Users/shivangisareen/Library/Caches/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Running setup.py bdist_wheel for future ... done
  Stored in directory: /Users/shivangisareen/Library/Caches/pip/wheels/bf/c9/a3/c538d90ef17cf7823fa51fc701a7a7a910a80f6a405bf15b1a
Successfully built pyLDAvis future
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [10]:
!pip install matplotlib

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [11]:
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shivangisareen/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
import re   #regular expressions package 
import numpy as np
import pandas as pd
from pprint import pprint

In [13]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

2018-06-18 15:11:53,639 : INFO : 'pattern' package not found; tag filters are not available for English


In [14]:
#plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [15]:
#LDA is a topic model that considers each document as a collection of topics in a certain
#proportion. A topic is nothing but a collection of dominant keywords that are typical
#representatives. Just by looking at the keywords, one can identify what the topic is all about.

In [16]:
#Import the stop words and save them in stop_words
from nltk.corpus import stopwords
stop_words=stopwords.words('english')
stop_words.extend(['from','subject','re','edu','use'])

In [17]:
#import dataset
df=pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()

['rec.autos' 'comp.sys.mac.hardware' 'rec.motorcycles' 'misc.forsale'
 'comp.os.ms-windows.misc' 'alt.atheism' 'comp.graphics'
 'rec.sport.baseball' 'rec.sport.hockey' 'sci.electronics' 'sci.space'
 'talk.politics.misc' 'sci.med' 'talk.politics.mideast'
 'soc.religion.christian' 'comp.windows.x' 'comp.sys.ibm.pc.hardware'
 'talk.politics.guns' 'talk.religion.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
10,From: irwin@cmptrc.lonestar.org (Irwin Arnstei...,8,rec.motorcycles
100,From: tchen@magnus.acs.ohio-state.edu (Tsung-K...,6,misc.forsale
1000,From: dabl2@nlm.nih.gov (Don A.B. Lindbergh)\n...,2,comp.os.ms-windows.misc


In [21]:
#There are many emails, newline and extra spaces. Getting rid of them using regular expressions

#convert to list
data=df.content.values.tolist()

#remove emails
data=[re.sub('\S*@\S*\s?','', sent) for sent in data]

#remove newline characters
data=[re.sub('\s+',' ', sent) for sent in data]

#remove single quotes
data=[re.sub("\'","", sent) for sent in data]

pprint(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


In [22]:
#to further clean up the text, tokenize each sentence into a list of words, removing punctuations and unnecessary
#characters altogether using Gensim's simple_preprocess()

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  #deacc=True removes punctuation

data_words=list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


In [23]:
#creating bigram and trigram models. Eg, front_bumper, oil_leak, maryland_college_park
#Gensim's Phrases model can build and implement bigrams, trigrams etc.

#build the bigram and trigram models
bigram=gensim.models.Phrases(data_words, min_count=5, threshold=100) #higher the threshold, fewer the phrases

trigram=gensim.models.Phrases(bigram[data_words], threshold=100)

bigram_mod=gensim.models.phrases.Phraser(bigram)

trigram_mod=gensim.models.phrases.Phraser(trigram)

print(trigram_mod[bigram_mod[data_words[0]]])



2018-06-18 15:26:23,432 : INFO : collecting all words and their counts
2018-06-18 15:26:23,434 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2018-06-18 15:26:28,310 : INFO : PROGRESS: at sentence #10000, processed 2653199 words and 899219 word types
2018-06-18 15:26:28,957 : INFO : collected 971890 word types from a corpus of 3011222 words (unigram + bigrams) and 11314 sentences
2018-06-18 15:26:28,958 : INFO : using 971890 counts as vocab in Phrases<0 vocab, min_count=5, threshold=100, max_vocab_size=40000000>
/Users/shivangisareen/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
2018-06-18 15:26:28,962 : INFO : collecting all words and their counts
2018-06-18 15:26:28,964 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2018-06-18 15:26:45,846 : 

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


In [24]:
!pip install spacy

    100% |████████████████████████████████| 17.6MB 86kB/s eta 0:00:01
    100% |████████████████████████████████| 92kB 3.3MB/s ta 0:00:011
    100% |████████████████████████████████| 1.2MB 967kB/s ta 0:00:01
    100% |████████████████████████████████| 51kB 5.6MB/s eta 0:00:01
    100% |████████████████████████████████| 194kB 2.3MB/s ta 0:00:01
    100% |████████████████████████████████| 71kB 4.5MB/s eta 0:00:01
    100% |████████████████████████████████| 604kB 1.5MB/s ta 0:00:01
    100% |████████████████████████████████| 51kB 5.9MB/s eta 0:00:01
    100% |████████████████████████████████| 389kB 2.1MB/s ta 0:00:01
  Running setup.py bdist_wheel for spacy ... done
  Stored in directory: /Users/shivangisareen/Library/Caches/pip/wheels/fb/00/28/75c85d5135e7d9a100639137d1847d41e914ed16c962d467e4
  Running setup.py bdist_wheel for murmurhash ... done
  Stored in directory: /Users/shivangisareen/Library/Caches/pip/wheels/b8/94/a4/f69f8664cdc1098603df44771b7fec5fd1b3d8364cdd83f512
  Running s

In [25]:
import spacy #for lemmatization

In [ ]:
#remove stopwords, make bigrams and lemmatize

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[doc] for doc in texts]

def lemmatization(texts, allowed_postages=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out=[]
    for sent in texts:
        doc=nlp(" ", join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed postages])